Option chains
=======

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=12)

<IB connected to 127.0.0.1:7497 clientId=12>

Suppose we want to find the options on the SPY. There are two ways to do that:
* The old way by requesting contract details
* The new and faster way

So first the old way. It starts with an ambiguous Option contract and uses that
as a wildcard to get the details of all contracts that match:

** This will take a while **

In [2]:
option = Option('SPY', exchange='SMART')
cds = ib.reqContractDetails(option)

contracts = [cd.summary for cd in cds]

print(len(contracts))
contracts[0]

6978


Contract(secType='OPT', conId=217296235, symbol='SPY', lastTradeDateOrContractMonth='20181221', strike=100.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   181221C00100000', tradingClass='SPY')

So that's a few thousand contracts. Let's put in some restrictions to get this number down:

* Use only the first 3 expirations after today that are on a Friday
* Use only strike prices within +- 20 dollar of the current SPY market price
* Use only strike prices that are a multitude of 5 dollar

For the first restriction the expirations are filtered with an isFriday method,
made unique with set(), then sorted and finally the first 3 taken:

In [3]:
import datetime

def isFriday(date):
    y = int(date[0:4])
    m = int(date[4:6])
    d = int(date[6:8])
    dd = datetime.date(y, m, d)
    return dd.weekday() == 4 and dd > datetime.date.today()

expirations = sorted(set(c.lastTradeDateOrContractMonth for c in contracts
            if isFriday(c.lastTradeDateOrContractMonth)))[:3]

expirations

['20180511', '20180518', '20180525']

Hmmm... perhaps we could have just taken the next three Fridays?
But the number of contracts is going down nicely:

In [4]:
contracts = [c for c in contracts if c.lastTradeDateOrContractMonth in expirations]

print(len(contracts))

952


To get the current price, first create the SPY contract:

In [5]:
spy = Stock('SPY', 'ARCA')

ib.qualifyContracts(spy)

[Stock(conId=756733, symbol='SPY', exchange='ARCA', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY')]

Then get the ticker. Requesting a ticker can take up to 11 seconds.

In [6]:
[ticker] = ib.reqTickers(spy)

ticker

Ticker(contract=Stock(conId=756733, symbol='SPY', exchange='ARCA', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY'), time=datetime.datetime(2018, 5, 8, 12, 49, 44, 710750, tzinfo=datetime.timezone.utc), bid=266.42, bidSize=12, ask=266.43, askSize=3, last=266.43, lastSize=1, volume=4572, close=266.92, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

Apply the final two restrictions:

In [7]:
spyPrice = ticker.marketPrice()

contracts = [c for c in contracts if
        spyPrice - 20 < c.strike < spyPrice + 20 and
        c.strike % 5 == 0]

print(len(contracts))
print(contracts[0])

oldContracts = contracts  # remember for later

48
Contract(secType='OPT', conId=303842315, symbol='SPY', lastTradeDateOrContractMonth='20180518', strike=250.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   180518C00250000', tradingClass='SPY')


Finally we have a list of usable option contracts.

Okay so now the new and faster way:

In [8]:
chains = ib.reqSecDefOptParams(spy.symbol, '', spy.secType, spy.conId)

util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,NASDAQOM,756733,SPY,100,"{20180601, 20180720, 20180921, 20180511, 20180...","{346.0, 306.0, 307.0, 10.0, 15.0, 20.0, 25.0, ..."
1,PSE,756733,SPY,100,"{20180601, 20180720, 20180921, 20180511, 20180...","{346.0, 306.0, 307.0, 10.0, 15.0, 20.0, 25.0, ..."
2,BATS,756733,SPY,100,"{20180601, 20180720, 20180921, 20180511, 20180...","{346.0, 306.0, 307.0, 10.0, 15.0, 20.0, 25.0, ..."
3,AMEX,756733,SPY,100,"{20180601, 20180720, 20180921, 20180511, 20180...","{346.0, 306.0, 307.0, 10.0, 15.0, 20.0, 25.0, ..."
4,PEARL,756733,SPY,100,"{20180601, 20180720, 20180921, 20180511, 20180...","{346.0, 306.0, 307.0, 10.0, 15.0, 20.0, 25.0, ..."
5,NASDAQBX,756733,SPY,100,"{20180601, 20180720, 20180921, 20180511, 20180...","{346.0, 306.0, 307.0, 10.0, 15.0, 20.0, 25.0, ..."
6,ISE,756733,SPY,100,"{20180601, 20180720, 20180921, 20180511, 20180...","{346.0, 306.0, 307.0, 10.0, 15.0, 20.0, 25.0, ..."
7,MIAX,756733,SPY,100,"{20180601, 20180720, 20180921, 20180511, 20180...","{346.0, 306.0, 307.0, 10.0, 15.0, 20.0, 25.0, ..."
8,SMART,756733,SPY,100,"{20180601, 20180720, 20180921, 20180511, 20180...","{346.0, 306.0, 307.0, 10.0, 15.0, 20.0, 25.0, ..."
9,CBOE,756733,SPY,100,"{20180601, 20180720, 20180921, 20180511, 20180...","{346.0, 306.0, 307.0, 10.0, 15.0, 20.0, 25.0, ..."


We want the options that trade on SMART:

In [9]:
chain = next(c for c in chains if c.exchange == 'SMART')
chain

OptionChain(exchange='SMART', underlyingConId=756733, tradingClass='SPY', multiplier='100', expirations={'20180601', '20180720', '20180921', '20180511', '20180530', '20180615', '20180606', '20190621', '20180516', '20200320', '20180622', '20190118', '20180514', '20200117', '20181231', '20180629', '20181221', '20180611', '20201218', '20180525', '20190329', '20180529', '20180518', '20180817', '20190920', '20191220', '20180613', '20180604', '20190315', '20180509', '20180928', '20180523', '20180521', '20180608'}, strikes={346.0, 306.0, 307.0, 10.0, 15.0, 20.0, 25.0, 311.0, 30.0, 35.0, 40.0, 45.0, 50.0, 316.0, 55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 321.0, 85.0, 90.0, 95.0, 100.0, 105.0, 326.0, 110.0, 115.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 331.0, 135.0, 332.0, 140.0, 145.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 157.0, 158.0, 159.0, 160.0, 161.0, 162.0, 163.0, 164.0, 165.0, 166.0, 167.0, 168.0, 169.0, 170.0, 171.0, 172.0, 173.0, 174.0, 175.0

What we have here is a matrix of expirations x strikes. From this we can build all the contracts:

In [10]:
strikes = [strike for strike in chain.strikes if
        strike % 5 == 0 and
        spyPrice - 20 < strike < spyPrice + 20]
expirations = sorted(exp for exp in chain.expirations if isFriday(exp))[:3]
rights = ['P', 'C']

contracts = [Option('SPY', expiration, strike, right, 'SMART')
        for right in rights for expiration in expirations for strike in strikes]

ib.qualifyContracts(*contracts)

print(len(contracts))
print()
print(contracts[0])

48

Option(conId=312732244, symbol='SPY', lastTradeDateOrContractMonth='20180511', strike=250.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   180511P00250000', tradingClass='SPY')


Let's see if the new way ends up with the same contracts as the old way:

In [11]:
set(contracts) == set(oldContracts)

True

Yep. Now to get the market data for all options in one go:

In [12]:
tickers = ib.reqTickers(*contracts)

tickers[0]

Ticker(contract=Option(conId=312732244, symbol='SPY', lastTradeDateOrContractMonth='20180511', strike=250.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   180511P00250000', tradingClass='SPY'), ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[], bidGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=0.019999999552965164, pvDividend=0.0, gamma=None, vega=None, theta=None, undPrice=266.4), askGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=0.029999999329447746, pvDividend=0.0, gamma=None, vega=None, theta=None, undPrice=266.4), lastGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=None, pvDividend=0.0, gamma=None, vega=None, theta=None, undPrice=266.4), modelGreeks=OptionComputation(impliedVol=0.19302683646294894, delta=-0.00022208800605610332, optPrice=0.0002775947871500553, pvDividend=0.0, gamma=0.00016966183458047034, vega=0.0003108027426272837, theta=-0.0002775947871500553, undPrice=266.4))

The option greeks are available from the ``modelGreeks`` attribute, and if there is a bid, ask resp. last price available also from ``bidGreeks``, ``askGreeks`` and ``lastGreeks``. For streaming ticks the greek values will be kept up to date to the current market situation.